#Introduce

This notebook is created for the case using to load the Dataset (DIV2k) with difference key, and preprocessing image:


1.   Load the dataset with key.
2.   Preprocessing image.
3.   Seperate the dataset.



In [1]:
print('import libary...')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
  

import libary...


In [2]:
def load_dataset(key='div2k',include_validate=True):
  print('using tensorflow_datasets to load the dataset','\nloading dataset...')
  if include_validate:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )
  else:
    (train_ds, val_ds), metadata = tfds.load(
    key,
    split=['train'],
    with_info=True,
    as_supervised=True,
    )
  print(metadata)
  return (train_ds,val_ds),metadata

In [3]:
def normalize_img(image):
  print('Normalizes images: `uint8` -> `float32`')
  return tf.cast(image, tf.float32) /255

In [4]:
def configure_performan(ds,BUFFER_SIZE=1000,BATCH_SIZE=4):
  ds=ds.cache()
  ds=ds.shuffle(buffer_size=BUFFER_SIZE)
  ds=ds.batch(BATCH_SIZE)
  ds=ds.prefetch(buffer_size=tf.data.AUTOTUNE)
  return ds

In [5]:
(rain_ds,val_ds),infor=load_dataset('div2k/bicubic_x4')

using tensorflow_datasets to load the dataset 
loading dataset...
EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

...

  0%|          | 0/800 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0.incomplete3PW1DG/div2k-validation.tfrecord


  0%|          | 0/100 [00:00<?, ? examples/s]

Dataset div2k downloaded and prepared to /root/tensorflow_datasets/div2k/bicubic_x4/2.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='div2k',
    version=2.0.0,
    description='DIV2K dataset: DIVerse 2K resolution high quality images as used for the challenges @ NTIRE (CVPR 2017 and CVPR 2018) and @ PIRM (ECCV 2018)',
    homepage='https://data.vision.ee.ethz.ch/cvl/DIV2K/',
    features=FeaturesDict({
        'hr': Image(shape=(None, None, 3), dtype=tf.uint8),
        'lr': Image(shape=(None, None, 3), dtype=tf.uint8),
    }),
    total_num_examples=900,
    splits={
        'train': 800,
        'validation': 100,
    },
    supervised_keys=('lr', 'hr'),
    citation="""@InProceedings{Agustsson_2017_CVPR_Workshops,
    	author = {Agustsson, Eirikur and Timofte, Radu},
    	title = {NTIRE 2017 Challenge on Single Image Super-Resolution: Dataset and Study},
    	booktitle = {The IEEE Conference on Computer Vision and Pattern Recognition (CVPR) Workshops},


In [6]:

len(rain_ds)

800

In [7]:
sub_data=rain_ds.take(40)

In [ ]:
for i in sub_data.take(1):
  print(i[0].shape)
  print(i[1].shape)

(306, 510, 3)
(1224, 2040, 3)


In [8]:

def show(image, label):
  plt.figure(figsize=(len(image.numpy())/80,len(image.numpy()[0])/80))
  plt.imshow(image)
  plt.title(np.array(label))
  plt.axis('off')

In [ ]:
for i in sub_data.take(2):
    # print(len(i[0].numpy()))
  show(i[0],'Low resolution')
  show(i[1],'High resolution')

In [ ]:
for i in sub_data.take(1):
  print(i[1].numpy())  

In [9]:
sub_data=sub_data.map(lambda lr,hr: (normalize_img(lr) , normalize_img(hr)),num_parallel_calls=tf.data.AUTOTUNE)

Normalizes images: `uint8` -> `float32`
Normalizes images: `uint8` -> `float32`


In [ ]:
len(sub_data)

40

In [31]:
def visualize(original, augmented,type='lr'):
  plt.figure(figsize=(len(original.numpy())/80,len(original.numpy()[0])/80))
  plt.subplot(1,2,1)
  plt.title('Original image'+type)
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image'+type)
  plt.imshow(augmented)

In [12]:
g=tf.random.get_global_generator()

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([4944792448642314029])>

In [35]:

for i in sub_data.take(6):
  seed= g.make_seeds(2)[0]
    # print(len(i[0].numpy()))
  au=tf.image.stateless_random_contrast(i[0],0.8,1.2,seed)
  visualize(i[0],au,'lr')
  au=tf.image.stateless_random_contrast(i[1],0.8,1.2,seed)
  visualize(i[1],au,type='hr')
  # show(i[1],'High resolution')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in sub_data.take(6):
  seed= g.make_seeds(2)[0]
    # print(len(i[0].numpy()))
  au=tf.image.stateless_random_contrast(i[0],0.8,1.2,seed)
  visualize(i[0],au,'lr')
  au=tf.image.stateless_random_contrast(i[1],0.8,1.2,seed)
  visualize(i[1],au,type='hr')
  # show(i[1],'High resolution')


In [ ]:
seed = (2, 3)

In [ ]:
np.array([[2.2,3],[23,3]]).shape

(2, 2)